In [1]:
# Callback duration
#
# Get trace data using the provided launch file:
#    $ ros2 launch tracetools_analysis pingpong.launch.py
#    (wait a few seconds, then kill with Ctrl+C)
#
# (optional) convert trace data:
#    $ ros2 run tracetools_analysis convert ~/.ros/tracing/pingpong/ust
#
# OR
#
# Use the provided sample converted trace file, changing the path below to:
#    'sample_data/converted_pendulum'

In [2]:
path = 'sample_data/converted_pendulum'

In [3]:
import sys
# Assuming a workspace with:
#   src/tracetools_analysis/
#   src/ros2_tracing/tracetools_read/
sys.path.insert(0, '../../tracetools_analysis/tracetools_analysis/')
sys.path.insert(0, '../../ros2_tracing/tracetools_read/')
import datetime as dt

from bokeh.plotting import figure
from bokeh.plotting import output_notebook
from bokeh.io import show
from bokeh.layouts import row
from bokeh.models import ColumnDataSource
from bokeh.models import DatetimeTickFormatter
from bokeh.models import PrintfTickFormatter
import numpy as np
import pandas as pd

from tracetools_analysis.loading import load_file
from tracetools_analysis.processor.ros2 import Ros2Handler
from tracetools_analysis.utils.ros2 import Ros2DataModelUtil

In [4]:
# Process
events = load_file(path)
handler = Ros2Handler.process(events)
handler.data.print_data()

 [100%] [Ros2Handler]
====================ROS 2 DATA MODEL===================
Contexts:
                          timestamp    pid version
context_handle                                    
94824409566288  1581097795452766567  31824  0.2.12

Nodes:
                          timestamp    tid      rmw_handle                 name namespace
node_handle                                                                              
94824409574672  1581097795459722384  31824  94824410211696  pendulum_controller         /
94824412290272  1581097795469258785  31824  94824412939280      pendulum_driver         /

Publishers:
                            timestamp     node_handle      rmw_handle                             topic_name depth
publisher_handle                                                                                                  
140732709071176   1581097795459713665  94824409574672  94824410633232                                /rosout    10
94824411647064    158109779546268

      callback_object            timestamp duration intra_process
0      94597995339144  1581097795463168030    17032         False
1      94824411656584  1581097795475719775     9849          True
2      94824412285168  1581097796553526227  8077829         False
3      94824412292336  1581097796562516734    23392         False
4      94824414836416  1581097796563327131  4132172         False
5      94824414843584  1581097796568375624     6380         False
6      94824412285168  1581097796569173666   216321         False
7      94824412292336  1581097796570268940     8104         False
8      94824415023624  1581097796570431973   833823         False
9      94824415023624  1581097796571374258   110083         False
10     94824414836416  1581097796571511174   180965         False
11     94824415166984  1581097796571884573   442495          True
12     94824415023624  1581097796572437820   112854         False
13     94824415166984  1581097796572567066    23703          True
14     948

In [5]:
data_util = Ros2DataModelUtil(handler.data)

callback_symbols = data_util.get_callback_symbols()

#print(callback_symbols)

output_notebook()
psize = 450
colours = ['#29788E', '#DD4968', '#410967']

KeyError: 94824414836416

In [ ]:
# Plot durations separately
colour_i = 0
for obj, symbol in callback_symbols.items():
    owner_info = data_util.get_callback_owner_info(obj)
    if owner_info is None:
        owner_info = '[unknown]'

    # Duration
    duration_df = data_util.get_callback_durations(obj)
    starttime = duration_df.loc[:, 'timestamp'].iloc[0].strftime('%Y-%m-%d %H:%M')
    source = ColumnDataSource(duration_df)
    duration = figure(
        title=owner_info,
        x_axis_label=f'start ({starttime})',
        y_axis_label='duration (ms)',
        plot_width=psize, plot_height=psize,
    )
    duration.title.align = 'center'
    duration.line(
        x='timestamp',
        y='duration',
        legend=str(symbol),
        line_width=2,
        source=source,
        line_color=colours[colour_i],
    )
    duration.legend.label_text_font_size = '11px'
    duration.xaxis[0].formatter = DatetimeTickFormatter(seconds=['%Ss'])

    # Histogram
    dur_hist, edges = np.histogram(duration_df['duration'])
    duration_hist = pd.DataFrame({
        'duration': dur_hist, 
        'left': edges[:-1], 
        'right': edges[1:],
    })
    hist = figure(
        title='Duration histogram',
        x_axis_label='duration (ms)',
        y_axis_label='frequency',
        plot_width=psize, plot_height=psize,
    )
    hist.title.align = 'center'
    hist.quad(
        bottom=0,
        top=duration_hist['duration'], 
        left=duration_hist['left'],
        right=duration_hist['right'],
        fill_color=colours[colour_i],
        line_color=colours[colour_i],
    )

    colour_i += 1
    show(row(duration, hist))

In [ ]:
# Plot durations in one plot
earliest_date = None
for obj, symbol in callback_symbols.items():
    duration_df = data_util.get_callback_durations(obj)
    thedate = duration_df.loc[:, 'timestamp'].iloc[0]
    if earliest_date is None or thedate <= earliest_date:
        earliest_date = thedate

starttime = earliest_date.strftime('%Y-%m-%d %H:%M')
duration = figure(
    title='Callback durations',
    x_axis_label=f'start ({starttime})',
    y_axis_label='duration (ms)',
    plot_width=psize, plot_height=psize,
)

colour_i = 0
for obj, symbol in callback_symbols.items():
    duration_df = data_util.get_callback_durations(obj)
    source = ColumnDataSource(duration_df)
    duration.title.align = 'center'
    duration.line(
        x='timestamp',
        y='duration',
        legend=str(symbol),
        line_width=2,
        source=source,
        line_color=colours[colour_i],
    )
    colour_i += 1
    duration.legend.label_text_font_size = '11px'
    duration.xaxis[0].formatter = DatetimeTickFormatter(seconds=['%Ss'])

show(duration)